## Подготовка подвыборки датасета из Wikidata с транскрипцией
Цель данной программы - рассмотреть каждый объект базового датасета и проверить, есть ли словарная статья с транскрипцией этого объекта в онлайн-словаре БКРС. Если статья есть - объект вместе с транскрипцией добавляется в отдельный датасет.

In [1]:
import pandas as pd
import html
import urllib
import time
import re
import textdistance

### Код, позволяющий найти на сайте БКРС транскрипцию иероглифического слова
Инпут: иероглифическая строка

Аутпут: транскрипция (пиньинь)

In [3]:
def pinyin_extractor(zh):
    address = 'https://bkrs.info/slovo.php?'
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    val = {'ch':zh}
    data = urllib.parse.urlencode(val)
    #data = data.encode('utf-8') # data should be bytes
    address = address + data
    #respData 
    #print(respData[:1000])
    #try:
    req = urllib.request.Request(address, headers={'User-Agent':user_agent})
    resp = urllib.request.urlopen(req)
    html = resp.read().decode("utf-8", "strict")
    #with open('check.txt', 'w', encoding='utf-8') as f:
    #    f.write(html)
    print(html[30:60])
    res = re.search(r"<div class='py'>\n(.*?)\n</div>", html, flags=re.DOTALL) #<div class='py'>(.*?)</div>
    #print(res)
    if res == None:
        return 'NaN'
    else:
        pinyin = res.group(1)
    #except:
    #    print('fail')
    #    return 'NaN'
    pinyin = pinyin.lower()
    time.sleep(0.2)
    return pinyin

In [14]:
pinyin_extractor('莫斯科')

	<title>莫斯科</title>
	<meta nam


'mòsīkē'

### Загрузка исходного датасета

In [2]:
df = pd.read_csv('data_total_2.csv')
#df.head()
without_ukr = df['country'] != 'Ukraine'
df_no_ukr = df[without_ukr]
df_no_ukr = df_no_ukr.drop_duplicates(subset=['label_ru', 'label_zh'])

In [3]:
df_no_ukr.shape

(12822, 10)

### Обработка исходного датасета
Код проходится циклом по датасету и, если находит транскрипцию в словаре, записывает ее в файл data_with_translit_2.csv.

In [6]:
with open('data_with_translit_2.csv', 'w',encoding='utf-8') as src:
    src.write('label_zh;label_ru;translit\n')

In [ ]:
for index, row in df_no_ukr.iterrows():
    #print(row['label_zh'], row['label_ru'])
    print(index, row['label_zh'])
    try:
        pinyin_zh = pinyin_extractor(row['label_zh'])
        if pinyin_zh != 'NaN' and pinyin_zh != '_':
            with open('data_with_translit_2.csv', 'a', encoding='utf-8') as f:
                print('caught')
                print(row['label_zh'], pinyin_zh, row['label_ru'])
                f.write(row['label_zh'] + ';' + row['label_ru'] + ';' + pinyin_zh + '\n')
    except:
        print('something wrong with '+row['label_zh'])
    #if index % 100 == 0:
        print(index)
        time.sleep(3)

0 伊曼德拉
	<title>伊曼德拉</title>
	<meta na
1 奥涅加
	<title>奥涅加</title>
	<meta nam
caught
奥涅加 àonièjiā Онежское
2 汉泰
	<title>汉泰</title>
	<meta name
3 奥特拉德诺耶
	<title>奥特拉德诺耶</title>
	<meta 
caught
奥特拉德诺耶 àotèlādénuòyē Отрадное
4 绍托泽罗
	<title>绍托泽罗</title>
	<meta na
5 克烈季
	<title>克烈季</title>
	<meta nam
6 尼季利
	<title>尼季利</title>
	<meta nam
7 奎托
	<title>奎托</title>
	<meta name
9 贝加尔
	<title>贝加尔</title>
	<meta nam
caught
贝加尔 bèijiā’ěr Байкал
10 丘克恰吉尔斯科耶
	<title>丘克恰吉尔斯科耶</title>
	<met
11 沃德洛泽罗
	<title>沃德洛泽罗</title>
	<meta n
12 兴凯
	<title>兴凯</title>
	<meta name
13 拉多加
	<title>拉多加</title>
	<meta nam
caught
拉多加 lāduōjiā Ладожское
14 维什蒂蒂斯
	<title>维什蒂蒂斯</title>
	<meta n
15 维戈泽罗
	<title>维戈泽罗</title>
	<meta na
16 伊尔门
	<title>伊尔门</title>
	<meta nam
17 捷列茨科耶


### Полученный датасет

In [4]:
df_fin = pd.read_csv('data_with_translit_2.csv', delimiter=';')
df_fin.head()

,label_zh,label_ru,translit
0,奥涅加,Онежское,àonièjiā
1,奥特拉德诺耶,Отрадное,àotèlādénuòyē
2,贝加尔,Байкал,bèijiā’ěr
3,拉多加,Ладожское,lāduōjiā
4,伏尔加,Волго,fú’ěrjiā
